In [2]:
%pip install requests
%pip install pandas
%pip install beautifulsoup4
%pip install boto3
%pip install logging


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl (11.3 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)

[notic

## 동아일보

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
import os
import re
from datetime import datetime
from urllib.parse import urlparse

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

output_dir = './donga'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

categories = {
    '정치': 'https://www.donga.com/news/Politics',
    '경제': 'https://www.donga.com/news/Economy',
    '국제': 'https://www.donga.com/news/Inter',
    '사회_사건': 'https://www.donga.com/news/Society/Event', # 앞 분기 기준
    '경제_과학': 'https://www.donga.com/news/It/List', # 앞 분기 기준
    '사회': 'https://www.donga.com/news/Society',
    '스포츠': 'https://www.donga.com/news/Sports',
}

result_categories = {
    '정치': 'politics',
    '경제': 'economy',
    '국제': 'world',
    '사회_사건': 'accident',
    '경제_과학': 'science',
    '사회': 'society',
    '스포츠': 'sports',
}

# 현재 날짜의 오전 8시 (시간대는 +09:00 기준)
today = datetime.now().replace(hour=8, minute=0, second=0, microsecond=0, tzinfo=datetime.now().astimezone().tzinfo)

def scrap_article(article_url, category):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        page_category = soup.find('ol', class_='breadcrumb').find('li', class_='breadcrumb_item').find('a').get_text(strip=True)
        logging.info(f"page_category: {page_category}")
        
        logging.info(f"category: {category}")

        category_parts = category.split('_')
        
        # 카테고리가 아닌 글은 패스
        if page_category not in category_parts:
            logging.info(f"{article_url} 는 해당 카테고리가 아니어서 생략합니다. ")
            return None
    except AttributeError:
        logging.error(f"{article_url} 는 찾을 수 없습니다.")
        return None

    # 헤드라인 스크랩
    try:
        title = soup.find('section', {'class': 'head_group'}).find('h1').get_text(strip=True)
    except AttributeError:
        title = 'N/A'
        
    try:
        date_button = soup.find('button', {'aria-controls': 'dateInfo'})
        time_span = date_button.find('span', {'aria-hidden': 'true'})
        time = time_span.get_text(strip=True)  # "2024-09-30 21:44" 형태
        time = time.replace(" ", "T") + ":00+09:00"  # ISO 8601 형식으로 변환 (예: 2024-09-30T21:44:00+09:00)
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        # 광고, 이미지 등 제외하고 텍스트만 추출
        content_section = soup.find('section', class_='news_view')
        for tag in content_section(['figure', 'script', 'div', 'ul']):  # 불필요한 태그 제거
            tag.decompose()
        content = content_section.get_text(separator="\n", strip=True)
    except AttributeError:
        content = 'N/A'

    result_category = result_categories.get(category, 'unknown')

    return {'title': title, 'source_url': article_url, 'content': content, 'category': result_category, 'source_created_at': time, 'press_id': 2, 'press_name' : '동아일보'}

# 파일 생성
def save_article_as_json(article_data, date, file_idx):
    category = article_data.get('category')
    file_name = date + "-" + str(file_idx) + '.json'
    result_dir = output_dir + "/" + category + "/" + date + "-" + str(file_idx)
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)

    file_path = os.path.join(result_dir, 'source-' + file_name)
    
    # 단일 dataframe 생성
    single_article_df = pd.DataFrame([article_data])
    
    # json으로 변환
    single_article_df.to_json(file_path, orient='records', lines=True, force_ascii=False)
    logging.info(f"Article saved to {file_path}")

def check_for_new_articles(base_url, category):
    response = requests.get(base_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_count = 0
    cards = soup.find_all('article', class_='news_card')
    if category not in ['사회_사건', '경제_과학']:
        cards.reverse()
    for card in cards:
        if article_count >= 4:
            break

        link_tag = card.find('a', href=True)
        if link_tag and 'href' in link_tag.attrs:
            article_link = link_tag['href']
        else:
            logging.warning("article_link is None or missing 'href'.")
            continue
    
        logging.debug(f"Scraping article: {article_link}")

        result = scrap_article(article_link, category)

        if result is None:
            logging.error("result is None");
            continue

        # if result.get('category') not in ['accident', 'science']:
        # 'source_created_at'를 datetime 객체로 변환
        article_time = datetime.strptime(result.get('source_created_at'), '%Y-%m-%dT%H:%M:%S%z')
        # 오늘 오전 8시 이후가 아니면 continue
        if article_time < today:
            logging.info(f"{article_link} 은 오늘 오전 8시 이전 기사이므로 생략합니다.")
            continue
        
        print(result)
        article_count += 1
        date = re.match(r'^[^T]+', result.get('source_created_at')).group()
        save_article_as_json(result, date, article_count)

def scrap_all_categories():
    
    for category, base_url in categories.items():
        logging.info(f"category (base_url): {category} ({base_url})")
        check_for_new_articles(base_url, category)
        time.sleep(2)

scrap_all_categories()

INFO:root:category (base_url): 정치 (https://www.donga.com/news/Politics)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.donga.com/news/Politics/article/all/20241013/130203240/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics/article/all/20241013/130203240/1 HTTP/11" 200 None
INFO:root:page_category: 정치
INFO:root:category: 정치
INFO:root:Article saved to ./donga/politics/2024-10-13-1/source-2024-10-13-1.json
DEBUG:root:Scraping article: https://www.donga.com/news/Politics/article/all/20241013/130203317/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics/article/all/20241013/130203317/1 HTTP/11" 200 None
INFO:roo

{'title': '조국 “이재명 전화 받았다…내일 부산 금정 선거 지원”', 'source_url': 'https://www.donga.com/news/Politics/article/all/20241013/130203240/1', 'content': '“윤석열 정권 심판 대의에 복무하기 위해 가기로”\n조국 조국혁신당 대표는 13일 “어제 이재명 더불어민주당 대표의 전화를 받고 14일 부산 금정구청장 선거를 지원하기 위해 금정구를 방문하기로 했다”고 말했다.\n조 대표는 이날 자신의 페이스북에 이같이 밝혔다.\n그는 “저와 조국혁신당은 이번 재보궐선거 후보를 낼 때부터 부산 금정구청장 후보가 민주당 김경지 후보로 단일화되더라도 지원유세를 할 것이라고 수차례 공언해왔다”며 “혁신당은 후보단일화 이후 민주당의 부산 선거 전략에 맞춰 지원을 하려고 준비를 갖추고, 민주당의 요청을 기다리고 있었다”고 전했다.\n또 “투표가 며칠 안 남은 지금 영광과 곡성의 선거캠프에서는 큰 아쉬움을 표했지만, 윤석열 정권 심판이라는 대의에 복무하기 위해 흔쾌히 부산에 간다”고 강조했다.\n이어 “대신 당원 동지 여러분과 영광 곡성의 지지자께서는 제 몫까지 뛰어달라”며 “투표함을 다 열 때까지는 결과를 알 수 없는 치열한 경쟁이지만, 혁신당표 지방행정을 펼치고 평가받는 기회를 갖고 싶은 열망은 더 뜨거워진다”고 언급했다.\n(서울=뉴스1)', 'category': 'politics', 'source_created_at': '2024-10-13T09:23:00+09:00', 'press_id': 2, 'press_name': '동아일보'}
{'title': '北, 쓰레기풍선에 GPS 달았다…낙하지점 데이터 쌓은 듯', 'source_url': 'https://www.donga.com/news/Politics/article/all/20241013/130203317/1', 'content': '5월 이후 총 28차례 오물·쓰레기풍선 부양\n풍선 살포량 6000개 넘어…6억원 이상 소요\n

DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics/article/all/20241013/130203496/1 HTTP/11" 200 None
INFO:root:page_category: 정치
INFO:root:category: 정치
INFO:root:Article saved to ./donga/politics/2024-10-13-3/source-2024-10-13-3.json
DEBUG:root:Scraping article: https://www.donga.com/news/Politics/article/all/20241013/130203698/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Politics/article/all/20241013/130203698/1 HTTP/11" 200 None
INFO:root:page_category: 정치
INFO:root:category: 정치
INFO:root:Article saved to ./donga/politics/2024-10-13-4/source-2024-10-13-4.json


{'title': '신원식 “북 ‘평양 무인기’ 대남 협박은 내부 통제용…무시가 정답”', 'source_url': 'https://www.donga.com/news/Politics/article/all/20241013/130203496/1', 'content': '“북 체제, 위협 있어야 생존…MZ는 안믿어”\n야권에 “북 비난 안하면서 우리 군에 가혹”\n“북 자살 결심 않으면 전쟁 일으키지 못해”\n“김정은 정밀 고위력 ‘현무’에 공포 느낄것”\n신원식 국가안보실장은 13일 북한이 평양 상공 무인기를 놓고 군사적 협박 등 대남 위협을 이어가는 데 대해 “(북한)체제 위협을 확대시키고 강조해서 내부 통제를 하는데 더 이점이 있다고 판단한 것으로, 그만큼 북한 내부가 흔들린다는 방증”이라며 “북한의 저런 말에 대해서는 무시하는 것이 최고의 정답”이라고 밝혔다.\n신 안보실장은 이날 KBS ‘일요진단’에서 “그 말대로 대한민국에서 무인기를 보내서 삐라까지 뿌렸는데 몰랐다면 얼마나 수치겠나. 그런데 북한은 방공망이 뚫렸다고 해서 느끼는 손해보다 ‘대한민국이 북한을 위협하고 있기 때문에 정신을 차리고 강력하게 대비해야 된다’(고 판단한 것)”이라며 이같이 말했다.\n그는 김여정 북한 노동당 부부장의 “다시 발견되는 순간 끔찍한 참변” 발언 등 고강도 대응이 군사적 도발을 위한 ‘명분 쌓기’라는 분석에 대해 “명분 쌓기 겸, 그보다 중요한 게 북한이 흔들리고 있는 내부 통제로 다시 긴장을 고조시키는 것이라고 본다”고 답했다.\n이어 “북한은 체제 위협을 국민들이 인식해야 체제가 생존하는 묘한 역설적 관계”라며 “북한의 MZ세대는 더 이상 체제 위협이 있다는 걸 잘 안 믿는데 이번에 (무인기를) 체제 위협으로 가장시킨 것”이라고 했다.\n신 실장은 무인기 관련 군의 ‘확인 불가’ 입장이 적절하다고 본다며 “야당에 좀 아쉬운 게, 북한의 도발과 억지와 핵무장에 대해서는 제대로 된 비난이나 문제제기를 안 하면서 우리 국민을 보호하기 위한 군과 정부의 노력에 대해서는 너무

INFO:root:category (base_url): 경제 (https://www.donga.com/news/Economy)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Economy HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.donga.com/news/People/article/all/20241012/130202730/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/People/article/all/20241012/130202730/1 HTTP/11" 200 None
INFO:root:page_category: 사람속으로
INFO:root:category: 경제
INFO:root:https://www.donga.com/news/People/article/all/20241012/130202730/1 는 해당 카테고리가 아니어서 생략합니다. 
ERROR:root:result is None
DEBUG:root:Scraping article: https://www.donga.com/news/Economy/article/all/20241012/130202760/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Economy/article/all/2024101

...

{'title': '‘칭얼댄다고’ 2개월 영아에 ‘성인 감기약’ 먹여 사망케 한 친모', 'source_url': 'https://www.donga.com/news/Society/article/all/20241013/130203389/1', 'content': '금고 1년\n생후 2개월 영아가 잠을 자지 않고 칭얼댄다는 이유로 성인이 먹는 감기약과 수면유도제를 분유에 타 먹여 사망에 이르게 한 20대 친모가 금고형을 선고받았다.\n창원지법 형사5단독 이재원 부장판사는 과실치사 혐의로 재판에 넘겨진 친모 A 씨(29)와 친모의 지인 B 씨(35)에게 금고 1년을 각각 선고했다고 13일 밝혔다.\nA·B 씨는 지난 2022년 8월 경남 창원시 한 모텔에서 A 씨의 2개월된 영아 C 군에게 성인용 감기약과 수면유도제를 탄 분유를 먹인 후 엎드려 잠을 자게 해 사망에 이르게 한 혐의로 기소됐다.\nC 군은 부검 결과 1차적으로 약 성분을 원인으로, 2차적으로 진정작용이 있는 약 성분이 체내에 있는 상태에서 비구폐쇄성질식 때문에 사망에 이른 것으로 조사됐다.\nA·B 씨는 C 군이 잠을 자지않고 칭얼댄다는 이유로 분유에 약을 타 먹이고 엎드려 자게 한 것으로 나타났다.\nB 씨는 A 씨, A 씨의 친구이자 자신의 동거녀, C 군, 자신의 자녀와 모텔에서 투숙하던 중 C 군이 칭얼대며 잠을 자지 않자 A 씨에게 동의를 받고 약을 탄 분유를 먹였다.\nC 군이 약이 든 분유를 먹고도 칭얼대자 A 씨로부터 “엎드려 재워라”라는 말은 들은 B 씨가 C 군을 엎드려 잠을 자게 한 것으로 조사됐다.\nA·B 씨는 수사 초기 C 군에게 성인용 감기약을 먹인 사실을 숨기다 부검을 통해 약 성분이 검출되자 범행을 시인했다.\n이 부장판사는 “피고인들이 조금만 주의를 기울였다면 영아가 사망하는 결과를 초래하지 않았을 것으로 보여 죄책이 매무 무겁고, 뒤늦게 범행을 인정하는 등 범행 후의 정상도 좋지 않기에 실형의 선고가 불가피하다”고 양형 이유를 밝혔다.\n(창원=뉴스1)', 'cate

INFO:root:category (base_url): 스포츠 (https://www.donga.com/news/Sports)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Sports HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.donga.com/news/Sports/article/all/20241013/130203118/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Sports/article/all/20241013/130203118/1 HTTP/11" 200 None
INFO:root:page_category: 스포츠
INFO:root:category: 스포츠
INFO:root:https://www.donga.com/news/Sports/article/all/20241013/130203118/1 은 오늘 오전 8시 이전 기사이므로 생략합니다.
DEBUG:root:Scraping article: https://www.donga.com/news/Sports/article/all/20241013/130203122/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Sports/article/all/20241013/130203122/1 HTTP/11" 200 

{'title': '삼성-LG, 22년 만에 가을야구 격돌…75.8% 확률 누가 잡을까', 'source_url': 'https://www.donga.com/news/Sports/article/all/20241013/130203141/1', 'content': '2002년 KS 이후 첫 만남…오늘 대구서 1차전\n삼성 레예스-LG 최원태, 선발 투수 맞대결\n프로야구 삼성 라이온즈와 LG 트윈스가 22년 만에 포스트시즌에서 맞대결을 벌인다.\n삼성과 LG는 13일 대구 삼성라이온즈파크에서 2024 신한 쏠뱅크 KBO 포스트시즌 플레이오프(PO·5전3선승제) 1차전을 치른다.\n올 시즌 2위 자리를 놓고 치열한 경쟁을 벌였던 삼성과 LG는 한국시리즈(KS·7전4선승제)로 가는 길목에서 마주하게 됐다. 두 팀이 가을야구에서 맞붙는 것은 2002년 KS 이후 무려 22년 만이다. 당시 삼성이 LG를 시리즈 전적 4승 2패로 제압하며 KS 정상에 올랐다.\n이번 시즌 상대 전적에서는 8승 1무 7패로 삼성이 근소하게 우위를 점했다.\nLG를 따돌리고 2위 자리를 꿰차며 PO 직행에 성공한 삼성은 정규시즌이 끝난 뒤 충분한 휴식을 가졌고, 최근 연습경기와 자체 평가전을 통해 실전 감각을 끌어올렸다.\n삼성은 1차전 선발 투수로 데니 레예스를 내세운다. 올 시즌 26경기에서 144이닝을 투구하며 11승 4패 평균자책점 3.81을 작성했다. 올해 LG와 2경기에서는 1승 평균자책점 3.60을 기록했다.\n삼성은 외국인 에이스 코너 시볼드, 베테랑 좌완 백정현 등이 부상으로 이탈하며 마운드가 헐거워졌지만, 화끈한 타선의 공격력은 여전하다. 구자욱을 필두로 김영웅, 르윈 디아즈, 강민호, 박병호 등이 1차전부터 LG 투수진 공략에 나선다.\n준플레이오프(준PO·5전3선승제)에서 KT 위즈를 3승 2패로 꺾고 PO에 오른 LG는 하루 휴식을 취한 뒤 삼성과 1차전에 임한다. 삼성에 비해 체력이 떨어져 있으나 좋은 흐름 속에서 대구로 향한다.\nLG는 나흘 휴식 후 마운드는 오르

DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Sports/article/all/20241013/130203382/1 HTTP/11" 200 None
INFO:root:page_category: 스포츠
INFO:root:category: 스포츠
INFO:root:Article saved to ./donga/sports/2024-10-13-3/source-2024-10-13-3.json
DEBUG:root:Scraping article: https://www.donga.com/news/Sports/article/all/20241013/130203504/1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Sports/article/all/20241013/130203504/1 HTTP/11" 200 None
INFO:root:page_category: 스포츠
INFO:root:category: 스포츠
INFO:root:Article saved to ./donga/sports/2024-10-13-4/source-2024-10-13-4.json


{'title': '‘논란 후 첫 국대’ 안세영, 감독·동료와 시간 차 출국…“할 말 없다”', 'source_url': 'https://www.donga.com/news/Sports/article/all/20241013/130203382/1', 'content': '‘작심발언’ 이후 첫 태극마크, 덴마크 오픈 출전\n전날 전국체전 결승 불참 “무릎 상태는 괜찮아”\n2024 파리 올림픽 이후 처음으로 태극마크를 달고 국제대회에 참가하는 ‘셔틀콕 여제’ 안세영(삼성생명)이 다소 어두운 표정으로 비행기에 몸을 실었다.\n감독을 비롯해 다른 동료들과는 다른 동선으로 출국장으로 들어가는 등 따로 움직이는 모습이었다. 출국 전날 제105회 전국체육대회 배드민턴 여자 일반부 단체전 결승에 불참한 것에 대해서는 “무릎 상태는 괜찮다”고 했다.\n안세영은 13일 인천국제공항을 통해 덴마크로 출국, 15일부터 덴마크 오덴세에서 열리는 ‘덴마크 오픈선수권’에 참가한다.\n파리 올림픽 배드민턴 여자 단식 금메달을 따낸 뒤 배드민턴계의 부조리에 대해 작심발언한 안세영은 이후 재활과 휴식에 집중하다가 전국체전을 통해 복귀했다. 그리고 곧바로 배드민턴 대표팀에 발탁돼 덴마크 오픈에 나서게 됐다.\n안세영은 두 달 만에 돌아온 전국체전에서 월등한 기량을 뽐냈다. 예선과 8강, 4강에서 한 게임도 내주지 않고 완승했다.\n하지만 12일 경남 밀양 배드민턴경기장에서 열린 김천시청과의 대회 결승전에서는 무릎 부상을 이유로 불참, 궁금증을 자아냈다.\n논란 후 말을 아껴온 안세영은 이날도 조심스러운 태도를 보였다.\n공항에서 만난 안세영은 무릎 상태를 묻는 질문에 “괜찮다”고 짧게 답했다. 이어 전국체전 결승을 뛰지 못한 이유에 대해서는 “드릴 말씀이 없다. 지금은 말을 할 수 있는 상황이 아니다”라고 모호하게 설명했다.\n안세영은 이날 대표팀 동료들과 별도의 동선으로 이동했다. 김학균 감독과 대표팀 선수단이 출국장을 통과한 뒤에도 관계자들과 남아있다가 약 1시간이 지나자 홀로 비행기를 탑승

## 중앙일보

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
import os
import re
from datetime import datetime
from urllib.parse import urlparse

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

output_dir = './joongang'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

categories = {
    '정치': 'https://www.joongang.co.kr/politics',
    '경제': 'https://www.joongang.co.kr/money',
    '국제': 'https://www.joongang.co.kr/world',
    '사회_사건': 'https://www.joongang.co.kr/society/accident', # 앞 분기 기준
    '경제_과학': 'https://www.joongang.co.kr/money/science', # 앞 분기 기준
    '사회': 'https://www.joongang.co.kr/society',
    '스포츠': 'https://www.joongang.co.kr/sports',
}


result_categories = {
    '정치': 'politics',
    '경제': 'economy',
    '국제': 'world',
    '사회_사건': 'accident',
    '경제_과학': 'science',
    '사회': 'society',
    '스포츠': 'sports',
}

# 현재 날짜의 오전 8시 (시간대는 +09:00 기준)
today = datetime.now().replace(hour=8, minute=0, second=0, microsecond=0, tzinfo=datetime.now().astimezone().tzinfo)

def scrap_article(article_url, category):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:

        page_category = soup.find('a', {'class': 'title'}).get_text(strip=True)
        logging.info(f"page_category: {page_category}")
        
        logging.info(f"category: {category}")

        category_parts = category.split('_')
        
        # 카테고리가 아닌 글은 패스
        if page_category not in category_parts:
            logging.info(f"{article_url} 는 해당 카테고리가 아니어서 생략합니다. ")
            return None
    except AttributeError:
        logging.error(f"{article_url} 는 찾을 수 없습니다.")
        return None

    # 헤드라인 스크랩
    try:
        title = soup.find('h1', {'class': 'headline'}).get_text(strip=True)
    except AttributeError:
        title = 'N/A'

    # 생성일 스크랩
    try:
        date = soup.find('p', {'class': 'date'})
        time = date.find('time').get('datetime')
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        content = soup.find('div', {'class': 'article_body'}).get_text(strip=True)
    except AttributeError:
        content = 'N/A'

    result_category = result_categories.get(category, 'unknown')

    return {'title': title, 'source_url': article_url, 'content': content, 'category': result_category, 'source_created_at': time, 'press_id': 1, 'press_name': '중앙일보'}

# 파일 생성
def save_article_as_json(article_data, date, file_idx):
    category = article_data.get('category')
    file_name = date + "-" + str(file_idx) + '.json'
    result_dir = output_dir + "/" + category + "/" + date + "-" + str(file_idx)
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)

    file_path = os.path.join(result_dir, 'source-' + file_name)
    
    # 단일 dataframe 생성
    single_article_df = pd.DataFrame([article_data])
    
    # json으로 변환
    single_article_df.to_json(file_path, orient='records', lines=True, force_ascii=False)
    logging.info(f"Article saved to {file_path}")

def check_for_new_articles(base_url, category):
    response = requests.get(base_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_count = 0
    cards = soup.find_all('li', class_='card')
    if category not in ['사회_사건', '경제_과학']:
        cards.reverse()
    for card in cards:
        if article_count >= 4:
            break

        link_tag = card.find('a', href=True)
        if link_tag and 'href' in link_tag.attrs:
            article_link = link_tag['href']
        else:
            logging.warning("article_link is None or missing 'href'.")
            continue
    
        logging.debug(f"Scraping article: {article_link}")

        result = scrap_article(article_link, category)

        if result is None:
            continue

        if result.get('category') not in ['accident', 'science']:
            # 'source_created_at'를 datetime 객체로 변환
            article_time = datetime.strptime(result.get('source_created_at'), '%Y-%m-%dT%H:%M:%S%z')
            # 오늘 오전 8시 이후가 아니면 continue
            if article_time < today:
                logging.info(f"{article_link} 은 오늘 오전 8시 이전 기사이므로 생략합니다.")
                continue

        print(result)
        article_count += 1
        date = re.match(r'^[^T]+', result.get('source_created_at')).group()
        save_article_as_json(result, date, article_count)

def scrap_all_categories():
    
    for category, base_url in categories.items():
        logging.info(f"category (base_url): {category} ({base_url})")
        check_for_new_articles(base_url, category)
        time.sleep(2)

scrap_all_categories()

INFO:root:category (base_url): 정치 (https://www.joongang.co.kr/politics)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /politics HTTP/11" 200 None
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25283672
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25283672 HTTP/11" 200 None
INFO:root:page_category: 정치
INFO:root:category: 정치
INFO:root:https://www.joongang.co.kr/article/25283672 은 오늘 오전 8시 이전 기사이므로 생략합니다.
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25283673
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25283673 HTTP/11" 200 None
INFO:root:page_category: 정치
INFO:root:category: 정치
INFO:root:https://www.joongang.co.kr/articl

{'title': '신원식 "야당, 북한 도발엔 비난않으면서 군과 정부엔 너무 가혹”', 'source_url': 'https://www.joongang.co.kr/article/25283844', 'content': "신원식 국가안보실장이 지난달 1일 용산 대통령실청사 브리핑룸에서 국가 사이버안보 기본계획 관련 브리핑을 하고 있다. 뉴스1신원식 국가안보실장은 13일 “북한이 자살을 결심하지 않을 것 같으면 전쟁을 일으키지 못할 것”이라고 밝혔다.신 실장은 이날 오전 KBS1 일요진단에 출연해 미국 외교 전문지 ‘포린 폴리시’가 “한반도에서 전쟁이 일어날 가능성이 1950년 한국전쟁 이후에 가장 높다”고 분석한 데 대해 “북한이 전쟁을 일으킬 가능성은 6·25 전쟁 이후에 늘 존재해 왔다”며 이같이 말했다.이어 “북한이 전쟁을 일으키지 못한 것은 북한의 마음에 달린 것이 아니라 우리의 의지와 태세에 달려 있다”며 “북한이 승산이 없도록 만들어주는 우리 국민의 단합된 노력이 중요하다”고 했다. 그러면서 \xa0“현재 한미동맹은 건전하고 대한민국은 선진국으로서 튼튼하다”고 말했다.신 실장은 북한의 한국 무인기 평양 침투주장과 관련해 ‘어떻게 보면 북한의 방공망이 뚫린 거 아니냐?’는 질문에 “평양 방공망이 뚫렸다고 해서 느끼는 손해보다 대한민국이 무인기를 보내서 북한을 위협하고 있기 때문에 다시 정신을 차리고 강력하게 대비해야 된다. 즉, 체제 위협을 확대시켜서, 강제로 해서 내부 통제를 하는데 더 이점이 있다고 판단한 것”이라고 말했다. 그러면서 “그만큼 북한 내부가 흔들린다는 방증을 오히려 보여준 것”이라고 했다.신 실장은 또 국방부의 '확인불가' 입장에 대해\xa0“확인해 줄 수 없다, 모든 책임은 북한에 있다, 우리 국민의 안전에 위해가 되면 강력하게 응징하겠다는 우리 군의 입장은 적절했다”고 했다.이러한 대응을 두고 야당에서 “정부 해명이 시시각각 변한다, 확인해 줄 수 없다 이 자체가 잘못된 거 아니냐”며 제기하는 비판에는 “북한의 행위에 대해 정부에게

DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25283844 HTTP/11" 200 None
INFO:root:page_category: 정치
INFO:root:category: 정치
INFO:root:Article saved to ./joongang/politics/2024-10-13-2/source-2024-10-13-2.json
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25283825
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25283825 HTTP/11" 200 None


{'title': '신원식 "야당, 북한 도발엔 비난않으면서 군과 정부엔 너무 가혹”', 'source_url': 'https://www.joongang.co.kr/article/25283844', 'content': "신원식 국가안보실장이 지난달 1일 용산 대통령실청사 브리핑룸에서 국가 사이버안보 기본계획 관련 브리핑을 하고 있다. 뉴스1신원식 국가안보실장은 13일 “북한이 자살을 결심하지 않을 것 같으면 전쟁을 일으키지 못할 것”이라고 밝혔다.신 실장은 이날 오전 KBS1 일요진단에 출연해 미국 외교 전문지 ‘포린 폴리시’가 “한반도에서 전쟁이 일어날 가능성이 1950년 한국전쟁 이후에 가장 높다”고 분석한 데 대해 “북한이 전쟁을 일으킬 가능성은 6·25 전쟁 이후에 늘 존재해 왔다”며 이같이 말했다.이어 “북한이 전쟁을 일으키지 못한 것은 북한의 마음에 달린 것이 아니라 우리의 의지와 태세에 달려 있다”며 “북한이 승산이 없도록 만들어주는 우리 국민의 단합된 노력이 중요하다”고 했다. 그러면서 \xa0“현재 한미동맹은 건전하고 대한민국은 선진국으로서 튼튼하다”고 말했다.신 실장은 북한의 한국 무인기 평양 침투주장과 관련해 ‘어떻게 보면 북한의 방공망이 뚫린 거 아니냐?’는 질문에 “평양 방공망이 뚫렸다고 해서 느끼는 손해보다 대한민국이 무인기를 보내서 북한을 위협하고 있기 때문에 다시 정신을 차리고 강력하게 대비해야 된다. 즉, 체제 위협을 확대시켜서, 강제로 해서 내부 통제를 하는데 더 이점이 있다고 판단한 것”이라고 말했다. 그러면서 “그만큼 북한 내부가 흔들린다는 방증을 오히려 보여준 것”이라고 했다.신 실장은 또 국방부의 '확인불가' 입장에 대해\xa0“확인해 줄 수 없다, 모든 책임은 북한에 있다, 우리 국민의 안전에 위해가 되면 강력하게 응징하겠다는 우리 군의 입장은 적절했다”고 했다.이러한 대응을 두고 야당에서 “정부 해명이 시시각각 변한다, 확인해 줄 수 없다 이 자체가 잘못된 거 아니냐”며 제기하는 비판에는 “북한의 행위에 대해 정부에게

INFO:root:page_category: 정치
INFO:root:category: 정치
INFO:root:https://www.joongang.co.kr/article/25283825 은 오늘 오전 8시 이전 기사이므로 생략합니다.
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25283828
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25283828 HTTP/11" 200 None
INFO:root:page_category: 정치
INFO:root:category: 정치
INFO:root:https://www.joongang.co.kr/article/25283828 은 오늘 오전 8시 이전 기사이므로 생략합니다.
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25283827
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25283827 HTTP/11" 200 None
INFO:root:page_category: 정치
INFO:root:category: 정치
INFO:root:https://www.joongang.co.kr/article/25283827 은 오늘 오전 8시 이전 기사이므로 생략합니다.
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25283827
DEBUG:url

...

{'title': '불혹 앞둔 호날두, A매치 133호골 폭발...포르투갈 네이션스리그 3연승', 'source_url': 'https://www.joongang.co.kr/article/25283840', 'content': "A매치 133호 골을 터뜨린 호날두. 로이터=연합뉴스불혹을 앞둔 수퍼스타 공격수 크리스티아누 호날두(39·포르투갈)가 A매치(국가대표팀 간 경기) 133호 골을 터뜨렸다.포르투갈은 13일(한국시간) 폴란드의 바르샤바 국립경기장에서 열린 2024~25시즌 유럽축구연맹(UEFA) 네이션스리그 리그A 1조 3차전에서 폴란드에 3-1로 이겼다. 대회 3연승을 달린 포르투갈은 1조 선두(승점 9)를 달렸다.호날두는 포르투갈이 1-0으로 앞서던 전반 37분 추가 골을 터뜨렸다.\xa0하프라인에서 공을 잡아 수비수 3명을 제치며 돌파하던 하파엘 레앙이 시도한 땅볼 슈팅이 오른쪽 골대를 맞고 나오자, 호날두가 왼발로 가볍게 밀어 넣었다. 폴란드가 후반 33분 피오트르 지엘린스키의 골로 한 점을 만회하면서 호날두의 골이 결승 골이 됐다. 이로써 그는 A매치 3경기 연속 골 행진도 이어갔다.지난해 유럽 무대를 떠나 사우디아라비아 알나스르로 이적한 호날두는 은퇴 수순을 밟을 거란 예상과 달리, 여전히 펄펄 날고 있다. 지금의 득점 페이스라면 지난 8월 자신의 소셜미디어(SNS)를 통해 밝힌 목표 '프로·A매치 통산 1000골' 달성도 가능할 것이라는 분석이 나온다. 호날두는 프로 통산 773골을 기록 중이다. A매치 득점 기록을 더하면 통산 904골이다.\xa02023~24시즌 알나스르에서만 50골을 넣은 그는 올 시즌도 소속팀에서 8골, A매치 3골 등 11골을 기록 중이다.리그A 4조에서는 스페인이 덴마크를 홈에서 1-0으로 물리치고 2연승을 기록했다. 승점 7을 쌓은 스페인은 덴마크(승점 6)를 제치고 조 선두로 올라섰다.피주영 기자 akapj@joongang.co.kr", 'category': 'sports', 'source_created_at':

INFO:root:page_category: 스포츠
INFO:root:category: 스포츠
INFO:root:Article saved to ./joongang/sports/2024-10-13-3/source-2024-10-13-3.json
DEBUG:root:Scraping article: https://www.joongang.co.kr/article/25283850
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25283850 HTTP/11" 200 None


{'title': "고개 숙인 '월드스타' 김예지…막판에 4점 전국체전 본선 탈락", 'source_url': 'https://www.joongang.co.kr/article/25283847', 'content': '전국체전에서 노메달에 그친 파리올림픽 스타 김예지. 연합뉴스"많이 아쉽다."전국체육대회 공기권총 본선에서 \'4점\'을 쏴 결선 진출이 불발된 2024 파리올림픽 은메달리스트 김예지(임실군청)가 아쉬움을 감추지 못했다.\xa0김예지는 13일 경남 창원국제사격장에서 열린 전국체전 사격 여자 일반부 10m 공기권총 본선에서 573점을 쐈다. 그는 전체 13위에 그치며\xa0상위 8명이 겨루는 결선 진출에 실패했다.김예지의 결승행이 좌절된 건 단 한 발의 실수 때문이다.\xa0공기권총 본선은 1시간 15분 동안 한 시리즈마다 10발씩 총 6시리즈(총 60발)을 쏴 순위를 가린다.\xa010.0점 이상은 10점으로, 9.0∼9.9점은 9점으로 처리되는 방식이다.\xa0김예지는 첫 시리즈에서 95점을 쐈다. 이후 2∼4시리즈에서 97점, 96점, 97점을 쏘며 차츰 순위를 끌어 올렸다.김예지는 5시리즈에서 9발까지 89점을 기록해 3위권 진입을 노렸다. 그러나 마지막 발에 4점을 쐈다. 6시리즈에서 95점을 쏴도 경쟁자와의 격차를 줄이기엔 어려웠다.\xa0사격 관계자는 "과녁의 12시 방향 4점에 꽂혔다. 팔을 위에서 내리면서 조준하다가 방아쇠를 잘못 당긴 것"이라고 설명했다.전국체전에서도 금메달을 차지한 파리올림픽 금메달리스트 오예진. 연합뉴스김예지는 경기 뒤 "마지막 시합을 좀 아쉽게 마쳤다. 많이 아쉽다"며 "좀 더 노력해서 좋은 모습을 보여드리도록 하겠다"고 짧게 소감을 밝혔다. 김예지는 전날 자신의 주종목인 25m 공기권총에서 7위, 공기권총 단체전에서도 5위에 머무르며 이번 전국체전은 빈손으로 돌아섰다. 파리올림픽 사격 10m 공기권총 금메달리스트 오예진은 결선에서 242.8점을 쏴 금메달을 목에 걸었다. 오예진은 앞서 열린 본선에서 585점을 

INFO:root:page_category: 스포츠
INFO:root:category: 스포츠
INFO:root:Article saved to ./joongang/sports/2024-10-13-4/source-2024-10-13-4.json


{'title': '염경엽 LG 감독 "손주영 2차전 대신 3차전 선발 기용"', 'source_url': 'https://www.joongang.co.kr/article/25283850', 'content': '선수단에게 박수를 보내는 LG 트윈스 염경엽 감독. 연합뉴스변화 대신 안정을 택했다. LG 트윈스가 준플레이오프 마지막 경기와 거의 같은 라인업을 꺼냈다. 손주영은 2차전 대신 3차전 선발로 나선다.LG는 13일 대구 삼성 라이온즈파크에서 열리는 삼성과의 플레이오프 1차전에 홍창기(우익수)-신민재(2루수)-오스틴 딘(1루수)-김현수(좌익수)-오지환(유격수)-문보경(3루수)-박동원(포수)-박해민(중견수)-문성주(지명타자) 타순을 내세웠다. 준PO 1차전부터 선발로 나간 9명의 야수가 그대로 나선다. 타순도 준PO 5차전과 비교하면 김현수와 오지환 순서만 바뀌었다.염경엽 LG 감독은 "김현수는 최근 타격감도 괜찮고, 상대전적도 좋다. 4번을 올리는 게 흐름상 좋을 것 같다"고 말했다. 문보경은 준PO 4차전까지 무안타를 치면서 타순이 4번에서 6번으로 조정됐다. 그러나 그는 5차전에서 첫 안타를 기록하면서 반등 조짐을 보이고 있다. 올 시즌 대구구장 성적(타율 0.357, 2홈런 7타점)도 좋았다. 염 감독은 "문보경이 분명히 살아날 거다. 선발 투수 최원태도 페넌트레이스 때 기복이 있었는데, 삼성전에서만은 항상 (공이 손에)긁혔다"며 웃었다.LG는 준플레이오프에서 대주자 요원 최원영을 제외하고, 김범석을 엔트리에 합류시켰다. 염 감독은 "상황이 되면 대타 요원 정도로 활용하려고 한다. KT보다는 왼손투수가 있고, 좌완에 대비하려고 한다. 컨디션이 괜찮다는 보고를 받았다"고 말했다.준플레이오프 5차전 데일리 MVP를 수상한 LG 손주영. 연합뉴스LG는 준PO에서 불펜으로 나섰던 손주영을 3차전 선발로 내보내기로 했다. 2차전 디트릭 엔스, 3차전 손주영, 4차전 임찬규로 이어지는 로테이션이 유력하다. 손주영은 준PO에서 7과 3분의 1이닝 동안 안타 3개

## 중앙일보 단일 기사 스크랩

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import logging
import os
import re

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

output_dir = './joongang'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def scrap_article(article_url):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 헤드라인 스크랩
    try:
        title = soup.find('h1', {'class': 'headline'}).get_text(strip=True)
    except AttributeError:
        title = 'N/A'

    # 생성일 스크랩
    try:
        date = soup.find('p', {'class': 'date'})
        time = date.find('time').get('datetime')
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        content = soup.find('div', {'class': 'article_body'}).get_text(strip=True)
    except AttributeError:
        content = 'N/A'

    return {
        'title': title,
        'source_url': article_url,
        'content': content,
        'category': 'sports',  # 이 부분은 해당 카테고리에 맞게 수정할 수 있습니다.
        'source_created_at': time,
        'press_id': 1,
        'press_name': '중앙일보'
    }

# 파일 생성
def save_article_as_json(article_data, date, file_idx):
    category = "sports"
    file_name = date + "-" + str(file_idx) + '.json'
    result_dir = output_dir + "/" + category + "/" + date + "-" + str(file_idx)
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)

    file_path = os.path.join(result_dir, 'source-' + file_name)
    
    # 단일 dataframe 생성
    single_article_df = pd.DataFrame([article_data])
    
    # json으로 변환
    single_article_df.to_json(file_path, orient='records', lines=True, force_ascii=False)
    logging.info(f"Article saved to {file_path}")

def scrap_single_categories():
    article_url = "https://www.joongang.co.kr/article/25284548"
    
    result = scrap_article(article_url)
    
    if result:
        date = re.match(r'^[^T]+', result.get('source_created_at')).group()  # 날짜 추출
        save_article_as_json(result, date, 1)
    else:
        logging.info(f"Failed to scrape article from {article_url}")

scrap_single_categories()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.joongang.co.kr:443
DEBUG:urllib3.connectionpool:https://www.joongang.co.kr:443 "GET /article/25284548 HTTP/11" 200 None
INFO:root:Article saved to ./joongang/sports/2024-10-16-1/source-2024-10-16-1.json


## 동아일보 단일 기사 스크랩

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
import os
import re
from datetime import datetime
from urllib.parse import urlparse

logging.basicConfig(level='DEBUG')

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

output_dir = './donga'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def scrap_article(article_url):
    response = requests.get(article_url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 헤드라인 스크랩
    try:
        title = soup.find('section', {'class': 'head_group'}).find('h1').get_text(strip=True)
    except AttributeError:
        title = 'N/A'
        
    try:
        date_button = soup.find('button', {'aria-controls': 'dateInfo'})
        time_span = date_button.find('span', {'aria-hidden': 'true'})
        time = time_span.get_text(strip=True)  # "2024-09-30 21:44" 형태
        time = time.replace(" ", "T") + ":00+09:00"  # ISO 8601 형식으로 변환 (예: 2024-09-30T21:44:00+09:00)
    except AttributeError:
        time = 'N/A'

    # 본문 스크랩
    try:
        # 광고, 이미지 등 제외하고 텍스트만 추출
        content_section = soup.find('section', class_='news_view')
        for tag in content_section(['figure', 'script', 'div', 'ul']):  # 불필요한 태그 제거
            tag.decompose()
        content = content_section.get_text(separator="\n", strip=True)
    except AttributeError:
        content = 'N/A'

    return {
        'title': title,
        'source_url': article_url,
        'content': content,
        'category': 'society',  # 이 부분은 해당 카테고리에 맞게 수정할 수 있습니다.
        'source_created_at': time,
        'press_id': 2,
        'press_name': '동아일보'
    }

# 파일 생성
def save_article_as_json(article_data, date, file_idx):
    category = "society"
    file_name = date + "-" + str(file_idx) + '.json'
    result_dir = output_dir + "/" + category + "/" + date + "-" + str(file_idx)
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)

    file_path = os.path.join(result_dir, 'source-' + file_name)
    
    # 단일 dataframe 생성
    single_article_df = pd.DataFrame([article_data])
    
    # json으로 변환
    single_article_df.to_json(file_path, orient='records', lines=True, force_ascii=False)
    logging.info(f"Article saved to {file_path}")

def scrap_single_categories():
    article_url = "https://www.donga.com/news/Society/article/all/20241016/130226096/1"
    
    result = scrap_article(article_url)
    
    if result:
        date = re.match(r'^[^T]+', result.get('source_created_at')).group()  # 날짜 추출
        save_article_as_json(result, date, 2)
    else:
        logging.info(f"Failed to scrape article from {article_url}")

scrap_single_categories()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.donga.com:443
DEBUG:urllib3.connectionpool:https://www.donga.com:443 "GET /news/Society/article/all/20241016/130226096/1 HTTP/11" 200 None
INFO:root:Article saved to ./donga/society/2024-10-16-2/source-2024-10-16-2.json
